# Transfer Learning

In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as f #for non-linear activation functions
from torchvision import datasets,transforms 
from torch.utils.data.sampler import SubsetRandomSampler #for splitting into validation, test set

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

1.) Perform classification on FashionMNIST, fashion apparels dataset, using a pre-
trained model which is trained on MNIST handwritten digit classification dataset.

In [6]:
# Import MNIST dataset 
trainset = datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

testset = datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

In [4]:
input_size = 784 # 28x28
hidden_size = 500 
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [7]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=trainset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=testset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

training the model

In [8]:
class CNNClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential( nn.Conv2d(1,64,kernel_size = 3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2,2), stride = 2),
                                 nn.Conv2d(64,128,kernel_size = 3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2,2), stride = 2),
                                 nn.Conv2d(128,64,kernel_size = 3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2,2), stride = 2),
                                )
        self.classification_head = nn.Sequential(
                                nn.Linear(64,20,bias = True),
                                nn.ReLU(),
                                nn.Linear(20,10,bias = True)
                                )
    def forward(self, x):
        features = self.net(x)
        return self.classification_head(features.view(batch_size, -1))

In [14]:
model = CNNClassifier().to(device)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
loss_fn = nn.CrossEntropyLoss()

#optimizer:
optim = optim.SGD(model.parameters(), lr = 0.001)

In [ ]:
# Train the model
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader): 
        
        # origin shape: [100, 1, 28, 28]
        # resized: [100, 784]
        #images = images.reshape(-1, 28*28).to(device)
        
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        # Backward and optimize
        optim.zero_grad()
        loss.backward()
        optim.step()

        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

### Now, using fashion dataset:

In [ ]:
#transformer to convert the images to tensor and normalize
transform=  transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5),(0.5),)]) 
#mean and std has to be tuples

#load data: trainset and testset
ftrainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data', download = True, train = True, transform = transform)
ftestset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data', download = True, train = False, transform = transform)

In [ ]:
# Data loader
ftrain_loader = torch.utils.data.DataLoader(dataset=ftrainset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

ftest_loader = torch.utils.data.DataLoader(dataset=ftestset, 
                                          batch_size=batch_size, 
                                          shuffle=False)